# Bereitstellen der Pakete und Funktionen
Führen Sie den unten stehenden Code mit **strg + Enter** aus um alle notwendigen Zusatzpakete und Funktionen verfügbar zu machen.

In [ ]:
# imported packages
import pandas as pd # used to handle csv- or excel files as a dataframe (table object)
import numpy as np # used for basic mathematical operations
import matplotlib.pyplot as plt # package for basic data plotting
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import seaborn as sns # additional package with more plotting options based on pyplot


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import os 
os.environ["OMP_NUM_THREADS"] = "1"



# from here code for more complexe plotting functions
def _resolve_column(df: pd.DataFrame, col):
    """Resolve a column specifier for flat or MultiIndex columns.
    - col can be None, a tuple (MultiIndex), or a string.
    - Exact matches are preferred. If not found and df has MultiIndex,
      try to find columns where any level equals the string.
    """
   
    if col is None:
        return None, None  # (series, display_name)
    # tuple (explicit MultiIndex)
    if isinstance(col, tuple):
        if col in df.columns:
            return df[col], " - ".join(map(str, col))
        raise KeyError(f"Column tuple {col} not found in DataFrame columns.")
    # string case
    # direct exact match (flat columns)
    if col in df.columns:
        return df[col], str(col)
    # if MultiIndex, try exact match on any level
    if isinstance(df.columns, pd.MultiIndex):
        # exact level match
        matches = [c for c in df.columns if any(str(level) == col for level in c)]
        if len(matches) == 1:
            return df[matches[0]], " - ".join(map(str, matches[0]))
        if len(matches) > 1:
            # prefer a match where top-level equals col
            top_matches = [c for c in matches if str(c[0]) == col]
            chosen = top_matches[0] if top_matches else matches[0]
            print(f"Multiple columns match '{col}', using {chosen}.")
            return df[chosen], " - ".join(map(str, chosen))
        # try substring match (e.g., user passes 'PC 1' and column is ('PCA','PC 1'))
        substr_matches = [c for c in df.columns if any(col in str(level) for level in c)]
        if len(substr_matches) >= 1:
            chosen = substr_matches[0]
            print(f"No exact match for '{col}', using first substring match {chosen}.")
            return df[chosen], " - ".join(map(str, chosen))
    # fallback: try substring in flat columns
    flat_sub = [c for c in df.columns if col in str(c)]
    if len(flat_sub) >= 1:
        chosen = flat_sub[0]
        print(f"No exact match for '{col}', using first flat substring match {chosen}.")
        return df[chosen], str(chosen)
    raise KeyError(f"Column '{col}' not found in DataFrame columns.")

def plot_dataframe(df: pd.DataFrame,
                   x,
                   y,
                   z=None,
                   hue=None,
                   figsize=(9, 6),
                   palette=None,
                   point_size=40,
                   alpha=0.9,
                   cmap="viridis",
                   title=None):
    """
    Flexible plotting for DataFrame: automatic 2D/3D scatter depending on z.
    x, y, z, hue can be strings or tuples (for MultiIndex). If z is None -> 2D.
    Returns (fig, ax).
    """
    # Resolve columns and display names
    X, x_label = _resolve_column(df, x)
    Y, y_label = _resolve_column(df, y)
    Z, z_label = _resolve_column(df, z) if z is not None else (None, None)
    H, h_label = _resolve_column(df, hue) if hue is not None else (None, None)

    if X is None or Y is None:
        raise ValueError("x and y must be provided and resolvable to DataFrame columns.")

    x_vals = X.values
    y_vals = Y.values
    z_vals = Z.values if Z is not None else None
    hue_vals = H.values if H is not None else None

    is_3d = z_vals is not None

    fig = plt.figure(figsize=figsize)
    if is_3d:
        ax = fig.add_subplot(111, projection='3d')
    else:
        ax = fig.add_subplot(111)

    # No hue: single color
    if hue_vals is None:
        color = sns.color_palette()[0]
        if is_3d:
            sc = ax.scatter(x_vals, y_vals, z_vals, s=point_size, alpha=alpha, color=color)
        else:
            sc = ax.scatter(x_vals, y_vals, s=point_size, alpha=alpha, color=color)
    else:
        # Determine whether hue is numeric or categorical.
        # If dtype is object or string-like -> categorical.
        if pd.api.types.is_numeric_dtype(H):
            # continuous hue
            if is_3d:
                # 3D scatter with continuous color: map to RGBA
                norm = plt.Normalize(np.nanmin(hue_vals), np.nanmax(hue_vals))
                cmap_obj = plt.get_cmap(cmap)
                colors = cmap_obj(norm(hue_vals))
                sc = ax.scatter(x_vals, y_vals, z_vals, c=colors, s=point_size, alpha=alpha)
                # create a ScalarMappable for colorbar
                mappable = plt.cm.ScalarMappable(norm=norm, cmap=cmap_obj)
                mappable.set_array(hue_vals)
                cbar = fig.colorbar(mappable, ax=ax, pad=0.1)
                cbar.set_label(h_label or str(hue))
            else:
                sc = ax.scatter(x_vals, y_vals, c=hue_vals, cmap=cmap, s=point_size, alpha=alpha)
                cbar = fig.colorbar(sc, ax=ax, pad=0.1)
                cbar.set_label(h_label or str(hue))
        else:
            # categorical hue (strings or objects)
            categories, uniques = pd.factorize(hue_vals)
            n_cats = len(uniques)
            if palette is None:
                palette = sns.color_palette(n_colors=n_cats)
            colors = [palette[i % len(palette)] for i in categories]
            if is_3d:
                sc = ax.scatter(x_vals, y_vals, z_vals, c=colors, s=point_size, alpha=alpha)
            else:
                sc = ax.scatter(x_vals, y_vals, c=colors, s=point_size, alpha=alpha)
            # legend
            handles = []
            for i, lab in enumerate(uniques):
                handles.append(plt.Line2D([], [], marker='o', color=palette[i % len(palette)], linestyle='', markersize=6))
            ax.legend(handles, uniques, title=(h_label or str(hue)), bbox_to_anchor=(1.05, 1), loc='upper left')

    # Labels and title
    ax.set_xlabel(x_label or str(x))
    ax.set_ylabel(y_label or str(y))
    if is_3d:
        ax.set_zlabel(z_label or str(z))
    if title:
        ax.set_title(title)

    plt.tight_layout()
    plt.show()
    return fig, ax


# Importieren der Daten
Importieren Sie mit dem unten stehenden Codeabschnitt _(Anpassungen des Codes sind nicht notwendig)_ die Daten aus dem CSV-Ordner in ein Pandas-DataFrame. Nach der Ausführung wird das DataFrame als Tabelle unterhalb des Codeblocks angezeigt. Die Daten bestehen aus 14 Spalten.

In [ ]:
#=== dont change paramter from here ===#
# creating data frame from nanoindendation data in csv folder
df = pd.read_csv("Data/CSV/OriginalData.csv", # file path
                header = [0,1], # number of rows for column heads
                sep = ";", # seperator between columns
                decimal = "," # decimal komma or decimal point
                )

# Set Indent column as Index
df.index = df[("Unnamed: 0_level_0","Unnamed: 0_level_1")]
df = df.drop(("Unnamed: 0_level_0","Unnamed: 0_level_1"), axis=1)

#visualization of the data structure

display(df)


# Plotten der mechanischen Daten
Mit der Ausführung des untenstehenden Codeblocks werden die mechanisch relevanten Daten als Mapping dargestellt.  
- Mit den `marker_options` und `colormap` können Sie die Darstellung des Plots beeinflussen.  
- Mit `indent_position = "real"` oder `"ideal"` legen Sie fest, ob die mechanischen Daten anhand ihrer Koordinaten aus der Rasterkraftmikroskopie oder aus den Daten der Nanoindentierung geplottet werden sollen.  
- Mit `data_left_plot` und `data_right_plot` können Sie festlegen, welche Spalten des DataFrames geplottet werden sollen.

**Frage:**  
Können Sie beim Vergleich beider Plots erkennen, welche Indents eine Gruppe bilden könnten?


In [ ]:
# === change parameter from here === #

#marker options
marker_shape = "s" #<-- "s" for squares, ">" for triangles
marker_size = 85 # setting the size of the markers
marker_tranparency = 0.75 # setting the transparency of the markers

#change the colormap of the plots
colormap = "crest_r" #<-- "viridis", "cividis", "inferno" ,"magma", "plasma", "rocket", "flare", "crest", "copper"

#choosing between ideal indent position and real indent positions
indent_position = "real" # <-- "real" or "ideal"

#selecting columns for plotting
data_left_plot = ("HARDNESS GPa","mean")
data_right_plot = ("MODULUS GPa","mean")
# ========================= #

#=== dont change paramter from here ===#

#image size
image_width_cm = 15 #change value to alter the image size
image_height_cm = 15 # change value to alter the image size

#Atomic Force Microscopy Mapping of Indentationmapping
background_image = plt.imread("Data/Images/BackGround.png")
dx,dy = -2.5,-3.2 #parameter for adjusting the background image
range_um = 50 # parameter for adjusting the background image

# automatical column selection for indent position
if indent_position == "real":
    x = ("x","real") # defining x axis
    y = ("y","real") # defining y axis
elif indent_position =="ideal":
    x = ("x","absolut") # defining x axis
    y = ("y","absolut") # defining x axis

#creating a figure object
fig, ax = plt.subplots(nrows = 1,
                       ncols = 2, #3 images next to each other
                       sharey=True)


fig.set_dpi(600) # increasing the resolution of the plot
fig.set_size_inches(image_width_cm/2.5,image_height_cm/2.54) #calcuating image size

# hardness plot 
sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = data_left_plot,
                ax = ax[0], #left plot
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap,as_cmap = True), # defining the color plaette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_tranparency , # transparence of the markers infill
               )

if indent_position == "real":
    ax[0].imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )

#modulus plot
sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = data_right_plot,
                ax = ax[1], #middle plot
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap,as_cmap = True), # defining the color plaette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_tranparency , # transparence of the markers infill
                )

if indent_position == "real":
    ax[1].imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )

for item in ax:
    item.set_aspect("equal") # ensure äquidistance on x and y axis
    sns.move_legend(loc = "lower center", bbox_to_anchor = (0.5,1), obj = item) # move legend above the corresponding plots


# Clustering nach KMeans
Durch Ausführen des folgenden Codeblocks werden die Daten des DataFrames analysiert und die Indents bezüglich ihrer mechanischen Kennwerte in Gruppen eingeteilt. Hierfür wird im folgenden Beispiel der KMeans-Algorithmus verwendet.

KMeans ist ein sogenannter **Clustering-Algorithmus**. Er wird verwendet, um Datenpunkte automatisch in Gruppen (sogenannte *Cluster*) einzuteilen – und zwar **ohne dass man vorher wissen muss, welche Gruppen es gibt**. Das macht KMeans zu einem typischen Verfahren des *unüberwachten Lernens*. Eine wichtige Vorgabe für den KMeans-Algorithmus ist die Anzahl der Cluster.

**Ablauf von KMeans**
1. Es werden zufällig Clusterzentren initialisiert.
2. Jeder Datenpunkt wird dem nächstgelegenen Zentrum zugewiesen.
3. Die Clusterzentren werden neu berechnet – als Mittelwert aller zugehörigen Punkte.
4. Die Schritte 2 und 3 wiederholen sich, bis sich die Clusterzuweisungen nicht mehr ändern oder ein Abbruchkriterium erreicht ist.

**Wichtige Parameter**
- `n_clusters` definiert die Anzahl der Gruppen, in die sich die Daten einteilen lassen.  
- `feature_set` definiert, welche Spalten des DataFrames für die Gruppierung berücksichtigt werden sollen.

**Hinweis**  
Stark korrelierende Daten (z. B. Härte in HV und Härte in GPa) sowie nicht physikalische Daten (x-, y-Koordinaten) können die Ergebnisse stark verzerren. Es ist ratsam, dem KMeans-Algorithmus nur für die Fragestellung _(Welche Phasen liegen vor?)_ relevante Daten zu übergeben.  
Nach der Ausführung des Codes wird dem DataFrame eine neue Spalte `("KMeans", "Label")` hinzugefügt. Diese enthält für jeden Indent die Information, zu welcher Gruppe er gehört.

**Nutzen Sie für die Übung `n_clusters = 4` oder höher.**


In [ ]:
# === change parameter from here ===
n_clusters = 4  # number of clusters
feature_set = df[[
                    ("HARDNESS GPa","mean"),
                    ("MODULUS GPa","mean"),
                    ("y","real"),
                    ("y","real")
                ]].copy()
#=========================#

#=== dont change paramter from here ===#


# using the feature set defind" 
X_kmeans = feature_set.copy().dropna()

# KMeans calculations 
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=42)
labels = kmeans.fit_predict(X_kmeans).astype(int)

# using strings for labelings instaed of numbers
#label_names = [f"Cluster {i+1}" for i in labels]
labels = pd.DataFrame(labels, index=X_kmeans.index)

# creating MultiIndex colum name for joining labels_df with df
#labels_df.columns = pd.MultiIndex.from_tuples([('KMeans', 'Label')])
labels.columns = pd.MultiIndex.from_tuples([('KMeans', 'Label')])
# deleting KMeans grouping if it already exists
if ('KMeans', 'Label') in df.columns:
    df = df.drop(columns=('KMeans', 'Label'))

# join KMeans Label_DataFrame with the original Dataframe df
df = df.join(labels)

display(df)

# Darstellen der KMeans-Ergebnisse
Der folgende Codeblock erzeugt ein Mapping, ähnlich zu den bereits zuvor erstellten Maps für die mechanischen Daten. Zusätzlich werden die über die Gruppen gemittelten Werte für Härte und E-Modul in einem weiteren Plot angezeigt.


In [ ]:
#=== change parameter from here ===#
#marker options
marker_shape = ">" #<-- "s" for squares, ">" for triangles
marker_size = 85 # setting the size of the markers
marker_tranparency = 0.75 # setting the transparency of the markers

#choosing between ideal indent position and real indent positions
indent_position = "real" # <-- "real" or "ideal"

#change the colormap of the plots
colormap = "tab10" #<-- "tab10", "tab20", "colorblind" 

#=========================#

#=== dont change paramter from here ===#
n_cluster_kmeans = df[("KMeans","Label")].nunique()+2
#image size
image_width_cm = 15 #change value to alter the image size
image_height_cm = 15 # change value to alter the image size

#Atomic Force Microscopy Mapping of Indentationmapping
background_image = plt.imread("Data/Images/BackGround.png")
dx,dy = -2,-3.2 #parameter for adjusting the background image
range_um = 50 # parameter for adjusting the background image

# automatical column selection for indent position
if indent_position == "real":
    x = ("x","real") # defining x axis
    y = ("y","real") # defining y axis
elif indent_position =="ideal":
    x = ("x","absolut") # defining x axis
    y = ("y","absolut") # defining x axis

#creating a figure object
fig, ax = plt.subplots(nrows = 1,
                       ncols = 1, 
                       sharey=False)


fig.set_dpi(600) # increasing the resolution of the plot
fig.set_size_inches(image_width_cm/2.5,image_height_cm/2.54) #calcuating image size

# Code for Mapping
sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = ("KMeans","Label"),
                ax = ax,
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap,n_cluster_kmeans)[2:], # defining the color plaette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_tranparency , # transparence of the markers infill
               )

ax.grid(False)
if indent_position == "real":
    ax.imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )

ax.set_aspect("equal") # ensure äquidistance on x and y axis
sns.move_legend(loc = "lower center", bbox_to_anchor = (0.5,1), obj = ax) # move legend above the corresponding plots

#Code for Catplot

cols = [
    ("HARDNESS GPa", "mean"),
    ("MODULUS GPa", "mean"),
    ("S2overP", "mean"),
    ("KMeans", "Label"),
]

df_sel = df[cols].copy()
df_sel.columns = ["Hardness", "Modulus", "S2overP", "Cluster"]

df_long = df_sel.melt(
    id_vars="Cluster",
    var_name="Property",
    value_name="Value"
)

g = sns.catplot(
        data=df_long,
        x="Cluster",
        y="Value",
        col="Property",      # drei Plots nebeneinander
        kind="violin",          # Mittelwerte als Balken
        sharey=False,
        hue = "Cluster", # unterschiedliche Skalen erlaubt
        palette = sns.color_palette(colormap,n_cluster_kmeans)[2:]
        )
g.figure.set_dpi(600)
g.figure.set_size_inches(20/2.5,7/2.54)
g._legend.remove()

for ax in g.axes.flat:
    ax.set_axisbelow(True)     
    ax.grid(True, linestyle="--", alpha=1,linewidth = 2)
    ax.tick_params(axis="x", rotation=45)
plt.show()

Es sollte zu erkennen sein, dass sich die mechanischen Eigenschften der einzelnen Gruppen kaum voneinander unterscheiden. Demnach ist die Qualität des KMeans-Clusterings zu hinterfragen.

Die nächsten Schritte befassen sich mit der Qualitätssteigerung von Cluster-Algorithmen.

# Principal Component Analysis (PCA)

Eine Möglichkeit, die Trennschärfe zwischen den Clustern zu erhöhen, ist die Durchführung einer **Principal Component Analysis** (PCA). Die PCA hilft dem Anwender bei der Beurteilung, welche Datenspalten _(also welches Feature-Set)_ an den Cluster-Algorithmus übergeben werden sollte. 

Eine weitere positive Eigenschaft der PCA ist die Reduzierung der Dimension des Feature-Sets. Je mehr Datenspalten an den Cluster-Algorithmus übergeben werden _(jede Spalte bedeutet eine zusätzliche Dimension)_, desto aufwendiger und ungenauer wird die Clusterbildung. Ab der vierten Dimension ist eine Visualisierung der Clusterbildung bereits nicht mehr möglich.  
Mit der PCA kann analysiert werden, wie viele Dimensionen zur Darstellung der Datenabhängigkeiten notwendig sind. Anschließend werden die Daten auf neue Achsen transformiert. Hierbei handelt es sich um ein Feature-Set mit reduzierter Dimensionalität, aber identischer Aussagekraft bezüglich der Datenabhängigkeiten.

Auch bei der PCA sollte man bei nicht physikalischen Größen (x-, y-Koordinaten) vorsichtig sein und sie besser nicht übergeben. Falls der Code im vorherigen Beispiel vor seiner Ausführung nicht kontrolliert wurde, wurden die Koordinaten dem KMeans-Algorithmus möglicherweise mit übergeben.

**Hinweis:**  
Physikalisch sinnvolle Materialkennwerte zur Phasentrennung sind zum Beispiel:  
Härte, E-Modul sowie die quadrierte Steifigkeit, aufgetragen über die Last.

- Mit `features` definieren Sie, welche Spalten an die PCA-Analyse übergeben werden sollen. Schließen Sie auf jeden Fall die x- und y-Koordinaten aus.  
- Mit `n_components` legen Sie fest, welche Dimension der reduzierte Datenraum haben soll. Lassen Sie `n_components` am besten auf 2 stehen.


In [ ]:
#=== change parameter ===#
#selecting important columns for clustering
features = df[[
                ("HARDNESS GPa","mean"),
                ("MODULUS GPa","mean"),
                ("S2overP","mean"),
            	("x","real"),
                ("y","real")
                ]].copy()

n_components = 2 #<-- determination of the dimension of the data space
#=========================#

#=== dont change paramter from here ===#

#deleting indents with not data (Indent 15, Indent 67) 
features.dropna(inplace = True)

# scale data very important for correct results!
scaler = StandardScaler() 
X_scaled = scaler.fit_transform(features)

# PCA on scaled data
pca = PCA(n_components=n_components) 
X_pca = pca.fit_transform(X_scaled)

fig = plt.figure(figsize=(8, 6)) 
if X_pca.shape[1] == 3:
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c='steelblue', s=40)
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')
    ax.set_title('PCA Scatterplot (3D)')
    plt.show()
    print("Erklärte Varianzanteile:", pca.explained_variance_ratio_)
    loadings = pd.DataFrame(pca.components_.T, columns=['PCA1', 'PCA2',"PCA3"], index=features.columns ) 
    print(loadings)

else:
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c='steelblue', s=40)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title('PCA Scatterplot (2D)')
    plt.show()
    print("Erklärte Varianzanteile:", pca.explained_variance_ratio_)
    loadings = pd.DataFrame(pca.components_.T, columns=['PC 1', 'PC 2'], index=features.columns ) 
    print(loadings)

# deleting PCA columns in orignial data dataframe if present 
if 'PCA' in df.columns.get_level_values(0):
    df = df.drop(columns='PCA', level=0)

# make dataframe for PCA1 and PCA2 and maybe PCA3 axis
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
pca_df = pd.DataFrame(X_pca, columns=pca_columns, index=features.index)

# transform to multi index column ("PCA","PC 1),("PC","PC 2"),...
pca_df.columns = pd.MultiIndex.from_product([['PCA'], pca_columns])

# join pca_df with orignal DataFrame df by Index "Indent Nr"
df = df.join(pca_df)

Mit den Varianzanteilen lässt sich überprüfen, ob die Reduzierung der Dimensionalität des ursprünglichen Feature-Sets zielführend ist. Wenn die neuen Achsen des reduzierten Feature-Sets _(PC1 und PC2)_ mehr als 80 % der Datenabhängigkeiten beschreiben ist eine Erhöhung der Dimension um eine weitere Achse _(PC3)_ meistens nicht mehr sinvoll. Eine Erhöhung von `n_components` auf 3 würde keinen zusätzlichen Mehrwert liefern und die Dimensionalität des Feature-Sets unnötig erhöhen. 

Die Tabelle darunter zeigt, welche Datenabhängigkeiten auf welcher Achse aufgetragen sind und welche Relevanz eine mechanische Kenngröße für diese Achse hat _(hohe Zahlen = hohe Relevanz)_. Kenngrößen, die auf jeder Achse einen Wert nahe 0 besitzen, können und sollten aus dem Feature-Set ausgeschlossen werden.

**Hinweis:**  

Sollten Sie mit den dargestellten Ergebnissen nicht zufrieden sein – zum Beispiel wegen  
- unnötiger PC-Achsen oder  
- unnötiger Materialkennwerte –,  

können Sie nachträglich `features` und `n_components` anpassen und den Code einfach erneut ausführen.


# Bestimmen einer geeigneten Clusteranzahl

Unüberwachte Cluster-Algorithmen wie **KMeans** oder das **Gaussian Mixture Model** benötigen die Angabe, wie viele Cluster erwartet werden. Doch nicht immer ist diese Anzahl bekannt _(vor allem, wenn man in den Daten auf der Suche nach Mustern ist)_.  

Der **Silhouette Score** und der **Elbow-Plot** sind beides Hilfsmittel, um eine geeignete Anzahl an Clustern zu definieren. Bei beiden Ansätzen handelt es sich um Kontrollen, die im Grunde erst nach dem Clustering angewendet werden.

Der folgende Codeblock führt nacheinander zehn KMeans-Algorithmen aus und erhöht in jedem Durchlauf die Clusteranzahl um 1. Anschließend werden für jede Variante der Silhouette Score und die Inertia _(Elbow-Plot)_ aufgezeichnet.

**Silhouette Score:**  

Der Silhouette Score bewertet, **wie gut ein Punkt zu seinem eigenen Cluster passt – im Vergleich zu anderen Clustern**. Er liegt zwischen –1 und +1:

- **Nahe +1:** Punkt ist gut im eigenen Cluster platziert und weit von anderen Clustern entfernt → gute Trennung.  
- **Nahe 0:** Punkt liegt zwischen zwei Clustern → keine klare Zuordnung.  
- **Negativ:** Punkt ist vermutlich im falschen Cluster → schlechte Struktur.

- **Interpretation:**  
  Ein hoher Silhouette Score (nahe 1) spricht für eine sinnvolle Clusterstruktur.

**Inertia (Elbow-Plot)**  

Die Inertia misst die durchschnittliche Entfernung der Punkte zu ihren Clusterzentren. Niedrige Werte zeigen kompakte Cluster, sinken jedoch automatisch bei steigender Clusteranzahl, sodass Inertia allein nicht zur Bestimmung der optimalen Clusteranzahl ausreicht.  
Im Elbow-Plot wird die Inertia gegen die Clusteranzahl aufgetragen; der „Knick“ _(falls überhaupt erkennbar)_ zeigt, ab wann zusätzliche Cluster nur noch eine geringe Verbesserung bringen.

**Hinweis:**  
Mit `feature_set` legen Sie fest, ob das dimensionsreduzierte PCA-Feature-Set oder das ursprüngliche Feature-Set verwendet werden soll. Da bereits eine PCA durchgeführt wurde, ist es empfehlenswert, `feature_set = "PCA"` zu wählen.


In [ ]:
#=== Change from here ===#
feature_set = "PCA" #<-- "PCA" or "features"
#========================#

#=== Dont change from here ===#
max_number_cluster = 10
silhouette_scores = []
inertias = []
ks = []

# Select data basis
if feature_set == 'PCA':
    if ("PCA","PC1") not in df.columns:
        raise ValueError(f"Für 'feature_set = PCA' bitte zuvor die PCA-Analyse (Code:Durchführen PCA) ausführen!")
    X = df['PCA'].dropna()
elif feature_set == 'features':
    X = features.copy()
else:
    raise ValueError("features_source must be 'PCA' or 'features'.")

for k in range(2,max_number_cluster):
    kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
    labels = kmeans.fit_predict(X)
    inertia = kmeans.inertia_
    inertias.append(inertia)
    ks.append(k)

# Silhouette Score is only defined for k > 1
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)
    print(f"k = {k}, Inertia = {inertia:.2f}, Silhouette Score = {score:.3f}")

# Plotting
fig, ax1 = plt.subplots(figsize=(9, 5))

color1 = 'tab:blue'
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia (Elbow)', color=color1)
ax1.plot(ks, inertias, marker='o', color=color1, label='Inertia')
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()  # second y-axis for Silhouette Score
color2 = 'tab:green'
ax2.set_ylabel('Silhouette Score', color=color2)
ax2.plot(ks, silhouette_scores, marker='s', linestyle='--', color=color2, label='Silhouette Score')
ax2.tick_params(axis='y', labelcolor=color2)

plt.title('KMeans: Elbow Plot & Silhouette Score')
fig.tight_layout()
plt.grid(True)
plt.show()


Im obigen Plot sind der Elbow-Plot und der Silhouette Score überlagert in Abhängigkeit von der Clusteranzahl dargestellt. Beurteilen Sie anhand des Plots, welche Clustergröße für den KMeans-Algorithmus am besten geeignet ist.

# Optimiertes Clustering nach KMeans

Führen Sie den untenstehenden Code aus. Als Feature-Set werden automatisch die PC1- und PC2-Spalten des DataFrames aus der PCA verwendet.  
Passen Sie `n_clusters` anhand der Auswertung des Silhouette Scores und des Elbow-Plots an.


In [ ]:
# === change parameter from here ===#
# KMeans Clustersize
n_clusters = 4  # number of clusters

# Plot Options
#marker options
marker_shape = ">" #<-- "s" for squares, ">" for triangles
marker_size = 85 # setting the size of the markers
marker_tranparency = 0.75 # setting the transparency of the markers

#choosing between ideal indent position and real indent positions
indent_position = "real" # <-- "real" or "ideal"

#change the colormap of the plots
colormap = "tab10" #<-- "tab10", "tab20", "colorblind" 
#=========================#

#=== dont change paramter from here ===#

#Code for KMeans
feature_pca = df.xs("PCA",axis=1,level=0).copy()
# using the feature set defind" 
X_kmeans = feature_pca.dropna()

# KMeans calculations 
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=42)
labels = kmeans.fit_predict(X_kmeans).astype(int)

# using strings for labelings instaed of numbers
#label_names = [f"Cluster {i+1}" for i in labels]
labels = pd.DataFrame(labels, index=X_kmeans.index)

# creating MultiIndex colum name for joining labels_df with df
#labels_df.columns = pd.MultiIndex.from_tuples([('KMeans', 'Label')])
labels.columns = pd.MultiIndex.from_tuples([('KMeans', 'Label')])
# deleting KMeans grouping if it already exists
if ('KMeans', 'Label') in df.columns:
    df = df.drop(columns=('KMeans', 'Label'))

# join KMeans Label_DataFrame with the original Dataframe df
df = df.join(labels)

#Code for plotting

n_cluster_kmeans = df[("KMeans","Label")].nunique()+2

#image size
image_width_cm = 15 #change value to alter the image size
image_height_cm = 15 # change value to alter the image size

#Atomic Force Microscopy Mapping of Indentationmapping
background_image = plt.imread("Data/Images/BackGround.png")
dx,dy = -2,-3.2 #parameter for adjusting the background image
range_um = 50 # parameter for adjusting the background image

# automatical column selection for indent position
if indent_position == "real":
    x = ("x","real") # defining x axis
    y = ("y","real") # defining y axis
elif indent_position =="ideal":
    x = ("x","absolut") # defining x axis
    y = ("y","absolut") # defining x axis

#creating a figure object
fig_map_kmeans, ax = plt.subplots(nrows = 1,
                       ncols = 1, 
                       sharey=False)


fig_map_kmeans.set_dpi(600) # increasing the resolution of the plot
fig_map_kmeans.set_size_inches(image_width_cm/2.5,image_height_cm/2.54) #calcuating image size

# Code for Mapping
map_kmeans = sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = ("KMeans","Label"),
                ax = ax,
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap,n_cluster_kmeans)[2:], # defining the color plaette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_tranparency , # transparence of the markers infill
               )

ax.grid(False)
if indent_position == "real":
    ax.imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )

ax.set_aspect("equal") # ensure äquidistance on x and y axis
sns.move_legend(loc = "lower center", bbox_to_anchor = (0.5,1), obj = ax) # move legend above the corresponding plots

#Code for Catplot

cols = [
    ("HARDNESS GPa", "mean"),
    ("MODULUS GPa", "mean"),
    ("S2overP", "mean"),
    ("KMeans", "Label"),
]

df_sel = df[cols].copy()
df_sel.columns = ["Hardness", "Modulus", "S2overP", "Cluster"]

df_long = df_sel.melt(
    id_vars="Cluster",
    var_name="Property",
    value_name="Value"
)

Cat_kmeans = sns.catplot(
        data=df_long,
        x="Cluster",
        y="Value",
        col="Property",      # drei Plots nebeneinander
        kind="violin",          # Mittelwerte als Balken
        sharey=False,
        hue = "Cluster", # unterschiedliche Skalen erlaubt
        palette = sns.color_palette(colormap,n_cluster_kmeans)[2:]
        )
Cat_kmeans.figure.set_dpi(600)
Cat_kmeans.figure.set_size_inches(20/2.5,7/2.54)
Cat_kmeans._legend.remove()

for ax in Cat_kmeans.axes.flat:
    ax.set_axisbelow(True)     
    ax.grid(True, linestyle="--", alpha=1,linewidth = 2)
    ax.tick_params(axis="x", rotation=45)
plt.show()

Betrachten Sie das Mapping und den Cluster-Vergleich darunter und vergleichen Sie die aktuellen Ergebnisse mit der unoptimierten KMeans-Analyse von zuvor.  
Es sollte auffallen, dass die Cluster nun eine höhere Trennschärfe besitzen als zuvor.


# Clustering nach Gaussian Mixture Model

Nachdem Sie die Clusterbildung mit KMeans kennengelernt und optimiert haben, wollen wir nun einen anderen Ansatz betrachten: das **Gaussian Mixture Model (GMM)**.  
Im Gegensatz zu KMeans, das die Cluster als gleichförmige Kugeln annimmt, geht GMM davon aus, dass die Daten aus **verschiedenen multivariaten Normalverteilungen** stammen. Dadurch können die Cluster **unterschiedliche Formen, Größen und Ausrichtungen** haben. Auch GMM gehört zum *unüberwachten Lernen* und erfordert die vorherige Festlegung der Clusteranzahl.  

Durch Ausführen des folgenden Codeblocks werden die Daten des DataFrames analysiert und die Indents bezüglich ihrer mechanischen Kennwerte in Gruppen eingeteilt. Hierfür wird im Beispiel der **GMM-Algorithmus** verwendet.  

**Ablauf von GMM**
1. **Initialisierung:** Für jedes Cluster wird eine multivariate Normalverteilung (Mittelwert & Kovarianz) festgelegt.  
2. **Wahrscheinlichkeitszuweisung:** Für jeden Datenpunkt wird die Wahrscheinlichkeit berechnet, zu jedem Cluster zu gehören.  
3. **Parameteraktualisierung:** Mittelwerte, Kovarianzen und Gewichtungen der Cluster werden angepasst, um die **Gesamtlikelihood** der Daten zu maximieren.  
4. **Iteration:** Schritte 2 und 3 wiederholen sich, bis die Parameter stabil bleiben oder ein Abbruchkriterium erreicht ist.  
5. **Clusterzuweisung:** Jeder Datenpunkt wird optional dem Cluster mit der höchsten Wahrscheinlichkeit zugewiesen.  

**Wichtige Parameter**
- `n_components`: Anzahl der Cluster  
- `feature_set`: Welche Spalten des DataFrames für die Gruppierung verwendet werden  

**Hinweis**  
Da GMM und KMeans unterschiedliche Ansätze verfolgen, ist es ratsam, für den GMM-Algorithmus die geeignete Clusteranzahl erneut zu bestimmen. **Dies wird im untenstehenden Codeblock ausgeführt.** Als Feature-Set wird das dimensionsreduzierte Feature-Set aus der PCA verwendet.

**Beurteilung der Clusteranzahl**  

Folgende Kriterien helfen bei der Entscheidung, wie viele Cluster sinnvoll sind:

**Silhouette Score**  
Siehe die Erklärung im Abschnitt „Bestimmung einer geeigneten Clusteranzahl“.  

**BIC & AIC**  
Informationskriterien, die Modellgüte und Komplexität abwägen:  
- Kleinerer Wert → besser  
- BIC bestraft komplexe Modelle stärker als AIC, daher bevorzugt BIC oft weniger Cluster  

Optimal ist eine Clusterzahl, bei der BIC/AIC minimal sind und der Silhouette Score hoch ist.


In [ ]:
# === Change from here === #
feature_set = "PCA" #<-- "PCA" or "features"
# ======================== #

silhouette_scores = []
neg_log_likelihoods = []
bic_scores = []
aic_scores = []
ks = []

# Select data basis
if feature_set == 'PCA':
    X = df['PCA'].dropna()
elif feature_set == 'features':
    X = features.copy()
else:
    raise ValueError("features_source must be 'PCA' or 'features'.")

# Compute metrics for different k
for k in range(2, max_number_cluster):
    gmm = GaussianMixture(n_components=k, random_state=42, n_init=5)
    gmm.fit(X)
    labels = gmm.predict(X)
    
    neg_ll = -gmm.score(X) * len(X)
    bic = gmm.bic(X)
    aic = gmm.aic(X)
    score = silhouette_score(X, labels)
    
    neg_log_likelihoods.append(neg_ll)
    bic_scores.append(bic)
    aic_scores.append(aic)
    silhouette_scores.append(score)
    ks.append(k)

# === Plot 1: -Log-Likelihood + Silhouette ===
fig, ax1 = plt.subplots(figsize=(9,5))

color1 = 'tab:purple'
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('− Log-Likelihood', color=color1)
ax1.plot(ks, neg_log_likelihoods, marker='o', color=color1, label='− Log-Likelihood')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.grid(True)

ax2 = ax1.twinx()
color2 = 'tab:green'
ax2.set_ylabel('Silhouette Score', color=color2)
ax2.plot(ks, silhouette_scores, marker='s', linestyle='--', color=color2, label='Silhouette Score')
ax2.tick_params(axis='y', labelcolor=color2)

# Combined legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.title('GMM: −Log-Likelihood & Silhouette Score')
fig.tight_layout()
plt.show()

# === Plot 2: BIC + AIC ===
plt.figure(figsize=(9,5))
plt.plot(ks, bic_scores, marker='x', linestyle='--', color='red', label='BIC')
plt.plot(ks, aic_scores, marker='^', linestyle='--', color='orange', label='AIC')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Information Criteria')
plt.title('GMM: BIC & AIC')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Betrachten Sie die Kriterien-Plots zur Bestimmung der geeigneten Clusteranzahl für den GMM-Algorithmus und bestimmen Sie die geeignete Clusteranzahl.  

Mit dem unteren Codeblock wird nun der GMM-Algorithmus ausgeführt, und die daraus resultierende Gruppierung in das DataFrame übernommen.

**Hinweis:**  
Vor der Ausführung des Codes überprüfen Sie, ob der Parameter `n_components` der korrekten Anzahl an GMM-Clustern entspricht, und ändern Sie den Wert gegebenenfalls.  

Die anschließend ausgeführten Plots bilden eine Gegenüberstellung der KMeans- und GMM-Algorithmen. Die Daten werden nach den Labeln in den Spalten `("KMeans","Label")` und `("GMM","Label")` sortiert und auf unterschiedliche Weise geplottet.  

Beurteilen Sie, wie relevant in diesem speziellen Fall die Auswahl des Cluster-Algorithmus ist:

- Welche Auswirkung hat die Auswahl des Algorithmus auf die Clusteranzahl?  
- Welchen Einfluss hat die Auswahl des Algorithmus auf die ermittelten mechanischen Eigenschaften der Cluster?


In [ ]:
# === change parameter from here ===#
# GMM Cluster number
n_components = 4  # number of clusters

# Plot Options
marker_shape = ">"  # "<" for triangles, "s" for squares
marker_size = 40
marker_transparency = 0.75

# choosing between ideal indent position and real indent positions
indent_position = "real"  # "real" or "ideal"

# colormap
colormap = "tab10"  # "tab10", "tab20", "colorblind"
#=========================#

#=== don't change from here ===#

# the allowed shape of the clusters
covariance_type = "full" #<-- full, tied, diag, spherical

# --- Prepare feature set ---
feature_pca = df.xs("PCA", axis=1, level=0).copy()
X_gmm = feature_pca.dropna()

# --- Fit GMM ---
gmm = GaussianMixture(n_components=n_components, random_state=42, n_init=5,covariance_type = covariance_type)
labels = gmm.fit_predict(X_gmm).astype(int)

# convert to DataFrame and MultiIndex column for joining
labels = pd.DataFrame(labels, index=X_gmm.index)
labels.columns = pd.MultiIndex.from_tuples([('GMM', 'Label')])

# remove existing GMM column if exists
if ('GMM', 'Label') in df.columns:
    df = df.drop(columns=('GMM', 'Label'))

# join labels with original df
df = df.join(labels)

# --- Scatterplot / Mapping ---

n_clusters_kmeans = df[("KMeans", "Label")].nunique() + 2
n_clusters_gmm = df[("GMM","Label")].nunique() + 2


image_width_cm = 15
image_height_cm = 15
background_image = plt.imread("Data/Images/BackGround.png")
dx, dy = -2, -3.2
range_um = 50

if indent_position == "real":
    x = ("x","real")
    y = ("y","real")
elif indent_position == "ideal":
    x = ("x","absolut")
    y = ("y","absolut")

fig_map, ax = plt.subplots(ncols=2,
                           nrows=1,
                          sharey=True)
fig_map.set_dpi(600)
fig_map.set_size_inches(image_width_cm/2.5, image_height_cm/2.54)

sns.scatterplot(
    data=df,
    x=x,
    y=y,
    hue=("KMeans","Label"),
    ax=ax[0],
    marker=marker_shape,
    palette=sns.color_palette(colormap,n_clusters_kmeans)[2:],
    s=marker_size,
    edgecolor=None,
    alpha=marker_transparency
)



if indent_position == "real":
    ax[0].imshow(
        background_image,
        extent=[dx, range_um+dx, dy, range_um+dy],
        aspect='equal',
        zorder=-1,
        origin="upper"
    )
ax[0].set_axisbelow(True)
ax[0].set_aspect("equal")

sns.scatterplot(
    data=df,
    x=x,
    y=y,
    hue=("GMM","Label"),
    ax=ax[1],
    marker=marker_shape,
    palette=sns.color_palette(colormap,n_clusters_gmm)[2:],
    s=marker_size,
    edgecolor=None,
    alpha=marker_transparency
)



if indent_position == "real":
    ax[1].imshow(
        background_image,
        extent=[dx, range_um+dx, dy, range_um+dy],
        aspect='equal',
        zorder=-1,
        origin="upper"
    )
ax[1].set_axisbelow(True)
ax[1].set_aspect("equal")
for item in ax:
    sns.move_legend(loc="lower center", bbox_to_anchor=(0.5,1), obj=item)
plt.show()



# Spalten für Properties
properties = [("HARDNESS GPa","mean"),("MODULUS GPa","mean"), ("S2overP","mean")]
n_props = len(properties)

# Figure mit 2 Zeilen (KMeans, GMM) und n_props Spalten
fig, axes = plt.subplots(nrows=2, ncols=n_props, figsize=(4*n_props, 8), sharey=False)

# Farbpalette
palette_kmeans = sns.color_palette(colormap,n_clusters_kmeans)[2:]
palette_gmm = sns.color_palette(colormap,n_clusters_gmm)[2:]

# --- Erste Zeile: KMeans ---
for i, prop in enumerate(properties):

    y = df[prop].dropna()
    sns.violinplot(
        data=df,
        x=df[("KMeans","Label")],
        y=y,
        hue=df[("KMeans","Label")],
        ax=axes[0, i],
        palette=palette_kmeans,
        split=False
    )
    axes[0, i].set_title(f"{prop} (KMeans)")
    axes[0, i].tick_params(axis="x", rotation=45)
    axes[0, i].set_xlabel("")
    axes[0, i].grid(True, linestyle="--", alpha=0.4)
    axes[0, i].set_axisbelow(True)   
    axes[0, i].legend_.remove()

# --- Zweite Zeile: GMM ---
for i, prop in enumerate(properties):
    y = df[prop].dropna()
    sns.violinplot(
        data=df,
        x=df[("GMM","Label")],
        y=y,
        hue=df[("GMM","Label")],
        ax=axes[1, i],
        palette=palette_gmm,
        split=False
    )
    axes[1, i].set_title(f"{prop} (GMM)")
    axes[1, i].tick_params(axis="x", rotation=45)
    axes[1, i].set_xlabel("")
    axes[1, i].grid(True, linestyle="--", alpha=0.4)
    axes[1, i].set_axisbelow(True)   
    axes[1, i].legend_.remove()

# Optional: globaler Titel
fig.suptitle("Cluster Properties: KMeans vs GMM", fontsize=16, y=1)

plt.tight_layout()
plt.show()

plot_dataframe(df, #<-- keep it 
               x=('PCA','PC1'), # choose x-axis
               y=("PCA",'PC2'), # choose y-axis
               z = None, # choose z-axis or set it to None for 2D plot
               hue=('KMeans','Label') # 
              ) 

plot_dataframe(df, #<-- keep it 
               x=('PCA','PC1'), # choose x-axis
               y=("PCA",'PC2'), # choose y-axis
               z = None, # choose z-axis or set it to None for 2D plot
               hue=('GMM','Label') # 
              ) 